- Link Model: https://huggingface.co/pitangent-ds/YOLOv8-human-detection-thermal
- Link Dataset: https://universe.roboflow.com/smart2/persondection-61bc2/dataset/5#
- Triển khai nhanh chóng trong tầm 3h với GPT4o, Kaggle, HuggingFace, Roboflow

In [ ]:
# Cài đặt các thư viện cần thiết cho YOLOv8
!pip install ultralytics supervision huggingface_hub

# 1. Data

Link: https://universe.roboflow.com/smart2/persondection-61bc2/dataset/
Chọn YoloV8 - Show download code

In [ ]:
!pip install roboflow

from roboflow import Roboflow

# Nhập API key của bạn
rf = Roboflow(api_key="R1Rsh0Bx3eiHdB9aSGr2")  # Thay thế bằng API key của bạn

# Kết nối đến workspace và project trên Roboflow
project = rf.workspace("smart2").project("persondection-61bc2")

# Chọn phiên bản project để tải dữ liệu
version = project.version(5)

# Tải dataset dưới định dạng YOLOv8
dataset = version.download("yolov8")



Vẽ cấu trúc thư mục sau khi down về bằng markdown
```bash
/kaggle/working/PersonDection-5/
│
├── valid/
│   ├── images/
│   └── labels/
│
├── train/
│   ├── images/
│   └── labels/
│
├── test/
│   ├── images/
│   └── labels/
│
├── data.yaml
├── README.dataset.txt
├── README.roboflow.txt
└── roboflow.zip
```

Kiểm tra dataset: 8000 imgs train, 1000 valid, 1000 test

# 2. Test Model Trước Khi Huấn Luyện (Inference)

## 2.1 Test 1 ảnh

In [ ]:
# Import các thư viện
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
import cv2 as cv

# Tải mô hình YOLOv8 đã được huấn luyện
model_path = hf_hub_download(
    repo_id="pitangent-ds/YOLOv8-human-detection-thermal",
    filename="model.pt"
)

# Khởi tạo mô hình từ file .pt đã tải
model = YOLO(model_path)


In [ ]:
# Hàm để chạy suy luận
def inference(image_path):
    # Đọc ảnh từ đường dẫn
    cv_image = cv.imread(image_path, cv.IMREAD_ANYCOLOR)
    
    # Chạy mô hình YOLO trên ảnh
    model_output = model(cv_image, conf=0.6, verbose=False)
    
    # Xử lý kết quả đầu ra
    detections = Detections.from_ultralytics(model_output[0])
    
    # Trả về kết quả suy luận
    return detections

# Chạy suy luận trên ảnh cụ thể
detections = inference('/kaggle/working/PersonDection-5/test/images/video-4FRnNpmSmwktFJKjg-frame-001044-YYpRCPPF5hNtz2bkA_jpg.rf.a1fcf98068d16a6ffd22e9e6d4fbd1e6.jpg')
print(detections)


- xyxy: Tọa độ của bounding box (477.36, 258.71, 505.37, 318.28). Đây là các tọa độ điểm giới hạn của hộp bao quanh đối tượng trong ảnh. 
    - x1, y1: Tọa độ của góc trên bên trái của hộp bao quanh (bounding box).
    - x2, y2: Tọa độ của góc dưới bên phải của hộp bao quanh.
    - Trong YOLO, gốc tọa độ (0, 0) thường nằm ở góc trên bên trái của hình ảnh, trục x nằm ngang từ trái sang phải và trục y nằm từ trên xuống dưới.
    - Trong hệ toạ độ Oxy Descartes, gốc tọa độ (0, 0) thường nằm ở giữa hệ trục, và trục y chạy từ dưới lên trên, trái ngược với YOLO.
- confidence: Độ tự tin của mô hình là 0.75238 (tức là 75.238%), cho thấy mức độ chắc chắn của mô hình về việc đối tượng được phát hiện là con người.
- class_id: 0 (tương ứng với lớp "HUMAN" trong mô hình của bạn).
- class_name: Tên lớp của đối tượng là HUMAN.


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Đọc ảnh
image = cv2.imread('/kaggle/working/PersonDection-5/test/images/video-4FRnNpmSmwktFJKjg-frame-001044-YYpRCPPF5hNtz2bkA_jpg.rf.a1fcf98068d16a6ffd22e9e6d4fbd1e6.jpg')

# Kiểm tra nếu ảnh được đọc thành công
if image is not None:
    # Lấy tọa độ bounding box
    x1, y1, x2, y2 = int(477.36), int(258.71), int(505.37), int(318.28)

    # Vẽ bounding box trên ảnh
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Màu xanh lá với độ dày 2px

    # Ghi nhãn đối tượng
    cv2.putText(image, "HUMAN", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Chuyển đổi từ BGR sang RGB để hiển thị với matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Hiển thị ảnh với matplotlib
    plt.imshow(image_rgb)
    plt.axis('off')  # Ẩn trục tọa độ
    plt.show()
else:
    print("Không thể đọc ảnh, vui lòng kiểm tra lại đường dẫn.")


# 3. Đánh giá model trước, trong, sau huấn luyện

### Tóm tắt quy trình đánh giá mô hình trước, trong và sau khi huấn luyện:

| **Giai đoạn**              | **Tập sử dụng**         | **Mục đích**                                                                                                  |
|----------------------------|-------------------------|---------------------------------------------------------------------------------------------------------------|
| **Trước khi training**      | Tập **validation**      | Đánh giá mô hình ban đầu để tinh chỉnh các **hyperparameters** mà không ảnh hưởng đến dữ liệu huấn luyện【66†source】【67†source】. |
| **Trong khi training**      | Tập **training** và **validation** | Mô hình học từ tập **training** và điều chỉnh tham số bằng cách đánh giá trên tập **validation**, nhằm tránh overfitting【60†source】【69†source】. |
| **Sau khi training xong**   | Tập **test**            | Đánh giá cuối cùng trên tập **test** để kiểm tra khả năng tổng quát hóa của mô hình với dữ liệu hoàn toàn mới【67†source】【69†source】. |

### **Lý do không đánh giá mô hình trên tập test trước khi huấn luyện**:

1. **Tránh dữ liệu rò rỉ (Data Leakage)**: Nếu đánh giá mô hình trên tập test trước khi huấn luyện, thông tin từ tập test có thể rò rỉ vào quá trình huấn luyện, khiến mô hình tối ưu hóa cho tập test và dẫn đến quá khớp (overfitting)【66†source】【69†source】.

2. **Tập test dành cho đánh giá cuối cùng**: Tập test được giữ lại để đánh giá khả năng tổng quát hóa của mô hình sau khi tất cả các bước huấn luyện và tinh chỉnh đã hoàn tất. Sử dụng tập test trước sẽ làm mất tính khách quan của đánh giá cuối cùng【67†source】【68†source】.

3. **Tập validation phù hợp cho việc tinh chỉnh**: Trong quá trình huấn luyện, tập validation được sử dụng để đánh giá và điều chỉnh các hyperparameter. Việc này giúp mô hình cải thiện mà không ảnh hưởng đến tính trung lập của tập test【69†source】.

4. **Đảm bảo kết quả đáng tin cậy**: Nếu tập test được giữ nguyên cho đến cuối cùng, kết quả đánh giá sẽ phản ánh đúng hiệu suất thực sự của mô hình trên dữ liệu chưa từng gặp【67†source】【66†source】.

### Tham khảo:
- [Training Set vs Validation Set vs Test Set](66)
- [How To Validate A Model In Machine Learning](68)
- [Train, Test, and Validation Sets](69)

Trong file `data.yaml`, 2 tập được sử dụng cho quá trình huấn luyện là:
- `train: ../train/images` (tập huấn luyện)
- `val: ../valid/images` (tập validation)

Tập test (`test: ../test/images`) sẽ không được sử dụng trong quá trình huấn luyện, mà chỉ dùng để đánh giá cuối cùng sau khi mô hình đã được huấn luyện xong.

- metrics_val: Đánh giá mô hình trước huấn luyện trên tập validation.
- model.train(): Huấn luyện mô hình với dữ liệu huấn luyện (train) và kiểm tra trên dữ liệu xác thực (val) để điều chỉnh tham số.
- metrics_test: Đánh giá mô hình sau huấn luyện trên tập test, đảm bảo rằng mô hình có khả năng tổng quát hóa tốt trên dữ liệu mới.

## 3.1 Đánh giá mô hình trước khi huấn luyện (trên tập validation):

- YOLOv8 đã có các cơ chế tích hợp sẵn để xử lý dữ liệu nhanh chóng và hiệu quả như sử dụng DataLoader của PyTorch, batching, pre-fetching, và caching. Bạn chỉ cần tinh chỉnh một số tham số như `workers` hoặc `cache` nếu muốn tăng hiệu suất. Không cần tự viết lại DataLoader trừ khi bạn muốn tùy chỉnh sâu hơn.

Dưới đây là bảng được mở rộng để giải thích chi tiết các cơ chế tối ưu hóa dữ liệu của **YOLOv8**:

| **Phương pháp**                   | **Giải thích**                                                                                                                                                                                                                                     | **Ví dụ sử dụng**                                                                                          |
|------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| **Batching**                       | Xử lý **nhiều hình ảnh trong một batch** thay vì xử lý từng ảnh riêng lẻ. Điều này giúp GPU hoặc CPU xử lý nhiều dữ liệu đồng thời, tận dụng tốt hơn tài nguyên phần cứng. Nó cũng giúp giảm overhead khi chuyển dữ liệu giữa CPU và GPU.  | Tự động trong YOLOv8 khi gọi hàm `train()`.                                                                 |
| **Pre-fetching**                   | **Pre-fetching** là quá trình nạp trước dữ liệu vào bộ nhớ trước khi cần thiết. Bằng cách đó, khi mô hình cần dữ liệu, dữ liệu đã sẵn sàng trong bộ nhớ, giảm thời gian chờ. Điều này giúp tránh **độ trễ I/O** khi nạp dữ liệu từ ổ cứng. | Được tích hợp sẵn trong `DataLoader` của YOLOv8. Không cần tùy chỉnh đặc biệt.                              |
| **Pinning memory**                 | **Pinning memory** là kỹ thuật lưu dữ liệu vào một vùng bộ nhớ không thể bị hoán đổi ra đĩa. Khi dữ liệu đã được "pinned", việc chuyển dữ liệu từ bộ nhớ CPU sang GPU sẽ nhanh hơn vì hệ thống không phải di chuyển dữ liệu qua các vùng nhớ khác. | Được hỗ trợ sẵn trong PyTorch và YOLOv8. Thường hoạt động khi `pin_memory=True` trong `DataLoader`.          |
| **Tăng số lượng worker**           | Sử dụng **nhiều worker** để nạp dữ liệu song song. Thay vì chỉ có một luồng đọc và nạp dữ liệu, có thể sử dụng nhiều worker để xử lý việc đọc dữ liệu đồng thời, giảm thời gian chờ đợi khi tải hình ảnh lớn hoặc dữ liệu nhiều.          | `model.train(data='data.yaml', epochs=50, imgsz=640, workers=4)`                                             |
| **Cache dữ liệu**                  | **Lưu dữ liệu vào bộ nhớ đệm (cache)** để giảm thời gian tải dữ liệu từ ổ cứng. Khi dữ liệu được lưu trong bộ nhớ đệm, các lần truy cập sau đó sẽ nhanh hơn, đặc biệt hữu ích khi làm việc với dữ liệu lớn mà không cần tải lại từ đĩa.    | `model.train(data='data.yaml', epochs=50, imgsz=640, cache=True)`                                            |
| **Mixed Precision Training (FP16)**| **Mixed precision** sử dụng cả **FP16 và FP32** trong quá trình huấn luyện. FP16 (16-bit floating point) giúp tăng tốc độ tính toán trên GPU bằng cách xử lý nhiều giá trị hơn mỗi chu kỳ, giảm yêu cầu bộ nhớ và tăng hiệu suất mà vẫn giữ độ chính xác với FP32 khi cần. | `model.train(data='data.yaml', epochs=50, imgsz=640, half=True)`                                             |



In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv8n đã được huấn luyện sẵn
model = YOLO("yolov8n.pt")

# Chạy đánh giá trên tập validation
metrics_val = model.val(data='/kaggle/working/PersonDection-5/data.yaml')  # Sử dụng tập validation

# In các chỉ số đánh giá trước khi huấn luyện
print("Đánh giá trước khi huấn luyện trên tập validation:")
print(f"Precision: {metrics_val.box.map50:.3f}")
print(f"Recall: {metrics_val.box.map:.3f}")
print(f"mAP@0.5: {metrics_val.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics_val.box.map:.3f}")


Output sau khi run 
```bash
runs/
└── detect/
    ├── val/
    ├── val2/
    ├── val3/
    └── val4/
```

Dưới đây là bảng tóm tắt chi tiết các chỉ số đánh giá trước khi huấn luyện lại mô hình YOLOv8:

| **Chỉ Số**            | **Ý Nghĩa**                                                                                          | **Giá Trị**         |
|-----------------------|------------------------------------------------------------------------------------------------------|---------------------|
| **Precision**          | Tỉ lệ giữa số dự đoán đúng (true positives) và tổng số các dự đoán (true positives + false positives). Precision cao nghĩa là mô hình ít dự đoán sai. | `metrics.box.map50` |
| **Recall**             | Tỉ lệ giữa số dự đoán đúng và tổng số đối tượng thực sự có (true positives + false negatives). Recall cao nghĩa là mô hình phát hiện tốt các đối tượng. | `metrics.box.map`   |
| **mAP@0.5**            | Mean Average Precision tại ngưỡng IoU = 0.5. mAP đo lường độ chính xác của dự đoán khi yêu cầu độ chồng lấn giữa các hộp giới hạn (bounding boxes) là 50%. | `metrics.box.map50` |
| **mAP@0.5:0.95**       | Mean Average Precision trung bình qua nhiều ngưỡng IoU từ 0.5 đến 0.95 (với bước nhảy 0.05). Đây là chỉ số tổng hợp, phản ánh độ chính xác của mô hình qua nhiều mức yêu cầu khác nhau về độ chồng lấn. | `metrics.box.map`   |

### Giải thích:
- **Precision** và **Recall** là hai chỉ số cơ bản để đánh giá khả năng dự đoán đúng của mô hình.
- **mAP@0.5** và **mAP@0.5:0.95** là những chỉ số tổng hợp đo lường khả năng dự đoán chính xác qua các ngưỡng IoU khác nhau, trong đó mAP@0.5 thường được sử dụng rộng rãi trong các bài toán phát hiện đối tượng.

- Precision 0.526 và mAP@0.5 0.526 là mức độ khá trung bình, nhưng điều này chỉ là trước khi huấn luyện, có nghĩa là mô hình chưa được tối ưu và chỉ dựa trên mô hình YOLOv8 cơ bản.
- Recall 0.333 khá thấp, chỉ ra rằng mô hình hiện tại không phát hiện được nhiều đối tượng trong dữ liệu. Điều này rất phổ biến trước khi huấn luyện vì mô hình chưa học được nhiều thông tin từ dữ liệu cụ thể.
- mAP@0.5:0.95 0.333: Đây là một chỉ số quan trọng thể hiện mức độ tổng quát hóa của mô hình khi các yêu cầu về độ chồng lấn (IoU) càng khắt khe hơn. Với chỉ số thấp, mô hình trước huấn luyện khó có khả năng dự đoán chính xác khi các đối tượng trong ảnh bị chồng chéo phức tạp.

## 3.2 Huấn luyện mô hình trên tập training và validation:Huấn luyện mô hình trên tập training và validation:

### Bảng tóm tắt về thời gian huấn luyện và các phương pháp tăng tốc:

| **Yếu tố**                | **Dự tính/Khuyến nghị**                                                               |
|---------------------------|----------------------------------------------------------------------------------------|
| **Số lượng ảnh**           | 8000 ảnh huấn luyện, 1000 ảnh xác thực.                                                |
| **Thời gian huấn luyện**   | 50-250 phút (khoảng 1 đến 4 giờ) trên GPU Tesla T4 (50 epochs, imgsz=640x640).        |
| **Sử dụng mô hình nhẹ hơn**| Dùng YOLOv8n (nano) để giảm thời gian huấn luyện, nhưng có thể giảm độ chính xác.      |
| **Giảm kích thước ảnh**    | Giảm kích thước ảnh từ 640x640 xuống 416x416 để tăng tốc độ huấn luyện.               |
| **Giảm số lượng epochs**   | Thử huấn luyện với 30 epochs để rút ngắn thời gian và theo dõi kết quả sau đó.        |
| **Mixed Precision Training**| Sử dụng huấn luyện với precision hỗn hợp (`half=True`) để tăng tốc độ huấn luyện.     |


### Kaggle sau khi huấn luyện model được lưu vào đâu, ta nên log lại vào wandb hoặc mlFlow không?

Dưới đây là bảng tóm tắt về các phương pháp lưu mô hình trên Kaggle và so sánh giữa **W&B** và **MLflow** để log quá trình huấn luyện:

| **Phương pháp**                                | **Mô tả chi tiết**                                                                                                     |
|-------------------------------------------------|------------------------------------------------------------------------------------------------------------------------|
| **Lưu mô hình trên Kaggle**                    | Sau khi huấn luyện, mô hình được lưu trong thư mục `runs/` (ví dụ: `runs/train/exp/best.pt`). Bạn có thể tải mô hình xuống từ Kaggle bằng lệnh API. |
| **Thời gian lưu trên Kaggle**                  | Mô hình được lưu tự động sau khi hoàn thành huấn luyện. Nếu không lưu trữ, kết quả có thể bị mất sau khi session Kaggle kết thúc. |
| **Sử dụng W&B để log**                         | - Giúp theo dõi quá trình huấn luyện (loss, accuracy, mAP, etc.).<br> - Lưu trữ mô hình và cho phép so sánh các phiên bản mô hình khác nhau. <br> - Cung cấp giao diện trực quan để theo dõi quá trình huấn luyện. |
| **Sử dụng MLflow để log**                      | - Tương tự như W&B, giúp quản lý và lưu trữ mô hình. <br> - Dễ dàng log hyperparameters, metrics, và lưu các phiên bản mô hình khác nhau. |
| **Cách sử dụng W&B trong Kaggle**              | ```python<br> import wandb <br> wandb.init(project="tên_dự_án", entity="tên_người_dùng") <br> model.train(data='data.yaml', log_artifacts=True) ``` |
| **Cách sử dụng MLflow trong Kaggle**           | ```python<br> import mlflow <br> mlflow.start_run() <br> mlflow.log_param("epochs", 50) <br> mlflow.log_metric("mAP@0.5", metrics_val.box.map50) <br> mlflow.end_run() ``` |



### **So sánh giữa W&B và MLflow**

| **Tiêu chí**                   | **Weights & Biases (W&B)**                                                                                          | **MLflow**                                                                                                           |
|--------------------------------|---------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|
| **Giao diện người dùng (UI)**  | Giao diện trực quan, dễ dàng chia sẻ và theo dõi theo thời gian thực giữa các nhóm.                                | Giao diện đơn giản, dễ quản lý các phiên bản và mô hình nhưng ít trực quan hơn.                                      |
| **Khả năng log dữ liệu**       | Hỗ trợ log tất cả các thông số, biểu đồ, mô hình một cách chi tiết và theo thời gian thực.                        | Log hyperparameters, metrics và phiên bản mô hình, nhưng ít tính năng trực quan hơn W&B.                             |
| **Cộng đồng và tích hợp**      | Tích hợp tốt với nhiều framework (PyTorch, TensorFlow) và có cộng đồng lớn.                                        | Tích hợp tốt với Databricks và phổ biến trong doanh nghiệp nhờ khả năng quản lý mô hình.                             |
| **Tính năng**                  | Có các tính năng nổi bật như lưu checkpoints, visualizations, dễ dàng chia sẻ dự án.                               | Quản lý và lưu trữ phiên bản mô hình, phù hợp với môi trường doanh nghiệp và sản xuất.                               |
| **Tốc độ tích hợp**            | Dễ thiết lập và tích hợp nhanh chóng vào các dự án machine learning.                                               | Thiết lập dễ dàng nhưng ít trực quan hơn so với W&B.                                                                 |
| **Dùng miễn phí**              | Có phiên bản miễn phí với các tính năng cơ bản.                                                                    | Hoàn toàn miễn phí và mã nguồn mở.                                                                                  |
| **Môi trường hỗ trợ**          | Chạy tốt trên các nền tảng cloud, hỗ trợ đầy đủ trên môi trường online/server.                                    | Không hỗ trợ đầy đủ UI trên Kaggle, cần sử dụng **MLflow Tracking Server** hoặc chạy môi trường cục bộ để truy cập log và UI. |

### **Lưu ý khi sử dụng MLflow trên Kaggle**:
- **MLflow UI trên Kaggle**: Kaggle không hỗ trợ trực tiếp MLflow UI, nhưng bạn có thể tải xuống file log hoặc chạy môi trường cục bộ để xem chi tiết log từ MLflow.
- **Sử dụng MLflow Tracking Server**: Nếu bạn muốn log vào một MLflow tracking server từ xa, có thể cấu hình server qua URL:
   ```python
   mlflow.set_tracking_uri("http://your-mlflow-server")
   ```




#### **Kết luận**:
- **W&B** phù hợp nếu bạn cần một giao diện trực quan, dễ dàng chia sẻ kết quả và theo dõi quá trình huấn luyện theo thời gian thực.
- **MLflow** phù hợp hơn cho việc quản lý và theo dõi các phiên bản mô hình trong môi trường sản xuất, đặc biệt là trong các doanh nghiệp【67†source】【68†source】【69†source】.

Mình chọn: log vào MLflow vì mình chưa dùng bao giờ (W&B mình từng dùng rùi)

In [ ]:
!pip install wandb
!pip install -U ipywidgets # cập nhật từ 7.7.1 lên 8.0 

In [ ]:
import os
import wandb

# # Need Enter key
# wandb.login()

# # Đăng nhập bằng biến môi trường
# wandb.login(key=os.getenv("WANDB_API_KEY"))

# Đăng nhập trực tiếp vào WandB bằng API key
wandb.login(key="c8767797aae76cbcd389ff29929ace1ac3021161")

In [ ]:
# Tải mô hình YOLOv8n
model = YOLO("yolov8n.pt")

# Huấn luyện mô hình và tích hợp WandB với tên chi tiết
model.train(
    data='/kaggle/working/PersonDection-5/data.yaml',  # Đường dẫn tới file cấu hình dữ liệu
    epochs=3,                   # Số lượng epochs
    imgsz=640,                   # Kích thước ảnh
    project="ThermalHumanDetect_YOLO",     # Tên project trên WandB
    name="YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.01",  # name RUN, Tên chi tiết
    save_period=1,               # Lưu kết quả mỗi 1 epochs
    device="cuda",               # Sử dụng GPU
    half=True,                   # Mixed precision để tăng tốc độ huấn luyện
    lr0=0.01                     # Learning rate ban đầu là 0.01
)


```bash
Ultralytics 8.3.22 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/PersonDection-5/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=cuda, workers=8, project=ThermalHumanDetect_YOLO, name=YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0, copy_paste=0.0, copy_paste_mode=flip, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml, save_dir=ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012
Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012', view at http://localhost:6006/
Tracking run with wandb version 0.18.3
Run data is saved locally in /kaggle/working/wandb/run-20241025_064049-jidy5o7a
Syncing run YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012 to Weights & Biases (docs)
View project at https://wandb.ai/doanngoccuong_nh/ThermalHumanDetect_YOLO
View run at https://wandb.ai/doanngoccuong_nh/ThermalHumanDetect_YOLO/runs/jidy5o7a
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Scanning /kaggle/working/PersonDection-5/train/labels.cache... 7037 images, 668 backgrounds, 0 corrupt: 100%|██████████| 7037/7037 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /kaggle/working/PersonDection-5/valid/labels.cache... 887 images, 0 backgrounds, 0 corrupt: 100%|██████████| 887/887 [00:00<?, ?it/s]
Plotting labels to ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        1/3      2.29G      1.425      1.839      1.369         41        640: 100%|██████████| 440/440 [01:18<00:00,  5.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:05<00:00,  5.00it/s]
                   all        887       2124      0.578      0.583      0.583      0.319

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        2/3      2.22G      1.398      1.423      1.358         52        640: 100%|██████████| 440/440 [01:14<00:00,  5.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:05<00:00,  5.01it/s]
                   all        887       2124      0.749      0.659      0.744      0.416

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        3/3      2.25G      1.305      1.227      1.297         41        640: 100%|██████████| 440/440 [01:13<00:00,  5.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:05<00:00,  4.87it/s]
                   all        887       2124      0.807      0.713      0.808      0.499

3 epochs completed in 0.076 hours.
Optimizer stripped from ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012/weights/last.pt, 6.2MB
Optimizer stripped from ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012/weights/best.pt, 6.2MB

Validating ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012/weights/best.pt...
Ultralytics 8.3.22 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:06<00:00,  4.09it/s]
                   all        887       2124      0.808      0.713      0.808      0.499
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012
Loading widget...
Run history:

lr/pg0	▁█▁
lr/pg1	▁█▁
lr/pg2	▁█▁
metrics/mAP50(B)	▁▆█
metrics/mAP50-95(B)	▁▅█
metrics/precision(B)	▁▆█
metrics/recall(B)	▁▅█
model/GFLOPs	▁
model/parameters	▁
model/speed_PyTorch(ms)	▁
train/box_loss	█▆▁
train/cls_loss	█▃▁
train/dfl_loss	█▇▁
val/box_loss	█▅▁
val/cls_loss	█▄▁
val/dfl_loss	█▆▁

Run summary:

lr/pg0	0.00068
lr/pg1	0.00068
lr/pg2	0.00068
metrics/mAP50(B)	0.8082
metrics/mAP50-95(B)	0.49928
metrics/precision(B)	0.80845
metrics/recall(B)	0.71328
model/GFLOPs	8.194
model/parameters	3011043
model/speed_PyTorch(ms)	2.104
train/box_loss	1.30457
train/cls_loss	1.2274
train/dfl_loss	1.29697
val/box_loss	1.27087
val/cls_loss	1.04561
val/dfl_loss	1.24087

View run YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012 at: https://wandb.ai/doanngoccuong_nh/ThermalHumanDetect_YOLO/runs/jidy5o7a
View project at: https://wandb.ai/doanngoccuong_nh/ThermalHumanDetect_YOLO
Synced 5 W&B file(s), 0 media file(s), 10 artifact file(s) and 21 other file(s)
Find logs at: ./wandb/run-20241025_064049-jidy5o7a/logs
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ccbd7214430>


curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([     ...] 

fitness: 0.5301676030741032
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.49928])
names: {0: 'person'}
plot: True
results_dict: {'metrics/precision(B)': 0.8084457451683595, 'metrics/recall(B)': 0.713276836158192, 'metrics/mAP50(B)': 0.8081973767947674, 'metrics/mAP50-95(B)': 0.4992754059940293, 'fitness': 0.5301676030741032}
save_dir: PosixPath('ThermalHumanDetect_YOLO/YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.012')
speed: {'preprocess': 0.18920893061658306, 'inference': 2.026759785218868, 'loss': 0.0005698392194986613, 'postprocess': 1.2812764964485384}
task: 'detect'
```

#### YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.01 - kết quả thu được

| **Thông số/Kết quả**              | **Giá trị**                       |
|-----------------------------------|------------------------------------|
| **Precision (Độ chính xác)**      | 0.808 (80.8%)                     |
| **Recall (Khả năng phát hiện)**   | 0.713 (71.3%)                     |
| **mAP@0.5**                       | 0.808 (80.8%)                     |
| **mAP@0.5:0.95**                  | 0.499 (49.9%)                     |
| **Epoch 1** - Precision           | 0.578                             |
| **Epoch 1** - Recall              | 0.583                             |
| **Epoch 1** - mAP@0.5             | 0.583                             |
| **Epoch 1** - mAP@0.5:0.95        | 0.319                             |
| **Epoch 2** - Precision           | 0.749                             |
| **Epoch 2** - Recall              | 0.659                             |
| **Epoch 2** - mAP@0.5             | 0.744                             |
| **Epoch 2** - mAP@0.5:0.95        | 0.416                             |
| **Epoch 3** - Precision           | 0.808                             |
| **Epoch 3** - Recall              | 0.713                             |
| **Epoch 3** - mAP@0.5             | 0.808                             |
| **Epoch 3** - mAP@0.5:0.95        | 0.499                             |
| **Kích thước ảnh**                | 640                               |
| **Số lượng ảnh huấn luyện**       | 7037                              |
| **Số lượng ảnh kiểm tra**         | 887                               |
| **Box Loss (Epoch 3)**            | 1.305                             |
| **Class Loss (Epoch 3)**          | 1.227                             |
| **DFL Loss (Epoch 3)**            | 1.297                             |
| **Optimizer**                     | AdamW (lr=0.002)                  |
| **AMP (Automatic Mixed Precision)**| Được kích hoạt                     |



Dưới đây là bảng tóm tắt chi tiết các chỉ số đánh giá trước và sau khi huấn luyện mô hình YOLOv8:

| **Chỉ Số**        | **Ý Nghĩa**                                                                                          | **Thế nào là tốt?**                                                                                     | **Kỳ vọng với bài toán**                                                                                   | **Trước huấn luyện** | **Sau huấn luyện (3 epochs)** |
|-------------------|------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------|-----------------------|------------------------------|
| **Precision**      | Tỉ lệ giữa số dự đoán đúng (true positives) và tổng số các dự đoán (true positives + false positives). Precision cao nghĩa là mô hình ít dự đoán sai. | Precision cao thể hiện độ chính xác của mô hình trong việc đưa ra dự đoán đúng.                          | Precision cao giúp giảm số lượng các dự đoán sai, đặc biệt trong các ứng dụng đòi hỏi độ chính xác cao.    | 0.526                 | 0.808                        |
| **Recall**         | Tỉ lệ giữa số dự đoán đúng và tổng số đối tượng thực sự có (true positives + false negatives). Recall cao nghĩa là mô hình phát hiện tốt các đối tượng. | Recall cao thể hiện khả năng phát hiện đầy đủ các đối tượng trong dữ liệu.                               | Kỳ vọng mô hình không bỏ sót các đối tượng quan trọng, đặc biệt trong các tình huống có nhiều đối tượng nhỏ hoặc ẩn. | 0.333                 | 0.713                        |
| **mAP@0.5**        | Mean Average Precision tại ngưỡng IoU = 0.5, đo lường độ chính xác của dự đoán khi yêu cầu độ chồng lấn giữa các hộp giới hạn là 50%. | mAP@0.5 cao cho thấy mô hình có khả năng phát hiện tốt các đối tượng với độ chính xác hộp giới hạn (bounding boxes). | Kỳ vọng mô hình đạt mAP@0.5 cao để tối ưu hóa khả năng phát hiện đối tượng trong các bài toán yêu cầu độ chính xác cao. | 0.526                 | 0.808                        |
| **mAP@0.5:0.95**   | Mean Average Precision trung bình qua nhiều ngưỡng IoU từ 0.5 đến 0.95. Chỉ số này phản ánh khả năng dự đoán chính xác với các mức độ chồng lấn khác nhau. | mAP@0.5:0.95 cao thể hiện khả năng tổng quát hóa tốt của mô hình, đặc biệt trong các bài toán có các đối tượng chồng lấn. | Kỳ vọng cải thiện mAP@0.5:0.95 để mô hình hoạt động tốt hơn khi đối tượng có các mức độ chồng lấn phức tạp.          | 0.333                 | 0.499                        |

### **Nhận xét**:
Mô hình đã cải thiện rõ rệt các chỉ số **precision**, **recall**, và **mAP@0.5** sau khi huấn luyện 3 epochs. Chỉ số **mAP@0.5:0.95** cũng đã tăng lên, tuy nhiên vẫn còn cần cải thiện thêm để đạt hiệu suất tốt hơn trong các tình huống đối tượng có mức độ chồng lấn cao.


| **Khái niệm** | **Chức năng**                                                                                              | **Trong code trên thì**                                                                                          |
|---------------|------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Project**   | Là nơi lưu trữ tất cả các lần huấn luyện (**run**) liên quan đến một dự án cụ thể.                           | `project="ThermalHumanDetect_YOLO"` là tên của dự án chứa tất cả các lần huấn luyện liên quan đến phát hiện người bằng ảnh nhiệt. |
| **Run**       | Là một phiên bản cụ thể của quá trình huấn luyện hoặc inference, lưu lại toàn bộ thông tin về lần chạy đó.    | `name="YOLOv8n_ThermalHumanDetect_img640_epochs5_mixedPrecision_noAug_lr0.01"` là tên của lần huấn luyện cụ thể. |
| **ID**        | Là mã định danh ngẫu nhiên do WandB gán tự động cho mỗi **run** để đảm bảo tính duy nhất.                   | Ví dụ: `jidy5o7a`. Đây là ID được WandB tự động tạo ra để nhận diện run duy nhất này.                            |
| **Artifact**  | Là tệp hoặc dữ liệu bạn lưu trữ lại, như trọng số mô hình hoặc tệp kết quả để quản lý và tái sử dụng.        | Artifact chưa được lưu rõ ràng trong đoạn mã, nhưng bạn có thể lưu trọng số mô hình (weights) sau khi huấn luyện. |
| **Model**     | Chính là mô hình học máy bạn đang huấn luyện hoặc inference, có thể được lưu lại như một artifact.           | Mô hình YOLOv8 được tải và huấn luyện: `model = YOLO("yolov8n.pt")`.                                             |
| **Run Table** | Bảng liệt kê thông tin từ nhiều lần **run** khác nhau để dễ dàng so sánh kết quả giữa các lần chạy.          | Bảng **Run Table** hiển thị các chỉ số từ nhiều lần chạy, không được trực tiếp đề cập trong code nhưng có thể xem trong WandB dashboard. |

### Giải thích:
- **ID** là mã duy nhất mà WandB tạo ra để đảm bảo không có hai run nào có cùng ID, ngay cả khi bạn đặt tên (**name**) cho nhiều lần chạy giống nhau.
- **Name** là tên bạn tự đặt, có thể trùng lặp hoặc tùy chỉnh để dễ nhận diện mục đích của run đó.
- **Run Table** là nơi bạn có thể xem tất cả các **run** trong một **project**, và so sánh các chỉ số như **loss**, **accuracy**, v.v., từ nhiều lần chạy khác nhau.

Bổ sung bảng tóm tắt các phiên bản của **YOLOv8**:

| **Phiên bản**  | **Tên đầy đủ**           | **Kích thước mô hình** | **Ưu điểm**                                        | **Nhược điểm**                       | **Ứng dụng**                                |
|----------------|--------------------------|------------------------|---------------------------------------------------|--------------------------------------|---------------------------------------------|
| **YOLOv8n**    | YOLOv8 Nano               | Nhỏ nhất                | Nhanh nhất, ít tài nguyên, phù hợp thiết bị nhúng | Độ chính xác thấp hơn                | Thiết bị di động, nhúng, edge devices       |
| **YOLOv8s**    | YOLOv8 Small              | Nhỏ                     | Cân bằng giữa tốc độ và độ chính xác              | Vẫn có thể thiếu độ chính xác        | Ứng dụng yêu cầu tốc độ nhanh, độ chính xác khá |
| **YOLOv8m**    | YOLOv8 Medium             | Trung bình              | Độ chính xác cao hơn YOLOv8n và YOLOv8s           | Cần nhiều tài nguyên hơn             | Dự án cần cân bằng giữa tốc độ và độ chính xác |
| **YOLOv8l**    | YOLOv8 Large              | Lớn                     | Độ chính xác cao, phù hợp với bài toán phức tạp   | Chạy chậm hơn, yêu cầu nhiều tài nguyên | Bài toán yêu cầu độ chính xác cao          |
| **YOLOv8x**    | YOLOv8 Extra-Large        | Lớn nhất                | Độ chính xác cao nhất, dùng cho bài toán phức tạp | Yêu cầu tài nguyên tính toán rất lớn | Ứng dụng yêu cầu độ chính xác rất cao      |

### Lưu ý:
- **YOLOv8n (nano)** là phiên bản **nhỏ nhất và nhanh nhất**, thích hợp cho các ứng dụng yêu cầu tốc độ cao với phần cứng hạn chế.
- **YOLOv8l (large)** và **YOLOv8x (extra-large)** là các phiên bản lớn hơn, mang lại độ chính xác cao hơn nhưng đòi hỏi nhiều tài nguyên tính toán hơn, thích hợp cho các ứng dụng cần độ chính xác tối đa.


Tóm tắt các lựa chọn mô hình YOLOv8 phù hợp cho **8000 ảnh train**, **1000 ảnh validation** và GPU **T4 x2** trên Kaggle:

| **Phiên bản YOLOv8** | **Ưu tiên**                      | **Độ chính xác**                 | **Thời gian dự kiến (50 epochs)** | **Khuyến nghị**                                                                                 |
|-----------------------|----------------------------------|-----------------------------------|-----------------------------------|-------------------------------------------------------------------------------------------------|
| **YOLOv8n (Nano)**    | Tốc độ tối đa, tài nguyên nhẹ   | Thấp nhất                         | < 1 giờ                           | Phù hợp cho các thử nghiệm nhanh hoặc các ứng dụng cần tốc độ cao, chấp nhận độ chính xác thấp. |
| **YOLOv8s (Small)**   | Cân bằng giữa tốc độ và độ chính xác | Tốt, độ chính xác vừa phải         | 1-2 giờ                           | Lựa chọn tốt nếu cần cân bằng giữa hiệu suất và thời gian huấn luyện.                           |
| **YOLOv8m (Medium)**  | Độ chính xác cao hơn YOLOv8s    | Cao                               | 2-4 giờ                           | Phù hợp cho các bài toán yêu cầu độ chính xác cao hơn và sẵn sàng đợi lâu hơn để huấn luyện.    |

### Gợi ý
- **YOLOv8s** là lựa chọn **cân bằng** giữa thời gian và độ chính xác cho bài toán của bạn.
- **YOLOv8m** nếu bạn cần **độ chính xác cao hơn** và có thể dành nhiều thời gian cho quá trình huấn luyện.

## 3.3 Đánh giá mô hình sau khi huấn luyện (trên tập test):

In [ ]:
# Đánh giá mô hình trên tập test
metrics_test = model.val(data='/kaggle/working/PersonDection-5/data.yaml', split="test")

# In các chỉ số sau huấn luyện
print("Kết quả đánh giá trên tập test sau huấn luyện:")
print(f"Precision: {metrics_test.box.map50:.3f}")
print(f"Recall: {metrics_test.box.map:.3f}")
print(f"mAP@0.5: {metrics_test.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics_test.box.map:.3f}")


## 3.4 ĐÁNH GIÁ 

Dưới đây là bảng so sánh giữa **Kaggle** và **WandB** với các tùy chọn `save_period`, `save=True`, và `wandb.log_artifact()` khi huấn luyện YOLOv8:

| **Nội dung lưu**            | **Kaggle**                                                                                                                                             | **WandB**                                                                                                                             |
|-----------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------|
| **Lưu kết quả mỗi epoch**   | , Kaggle lưu các chỉ số như `precision`, `recall`, `mAP` và các hình ảnh dự đoán (prediction images) sau mỗi epoch.                             | WandB tự động ghi nhận các chỉ số này lên giao diện trong mỗi epoch nhưng không lưu trọng số mô hình theo mặc định.                   |
| **Lưu trọng số mô hình (`.pt`)** mỗi 10 epochs (`save_period=10`, Mặc định `save=True`), | Kaggle lưu tệp trọng số mô hình `.pt` vào thư mục `weights` của project theo chu kỳ mỗi `save_period=10` epochs (ví dụ `epoch_10.pt`, `epoch_20.pt`). | WandB không tự động lưu trọng số mô hình vào mục Artifacts. Để lưu, bạn cần sử dụng `wandb.log_artifact()` theo cách thủ công【121†source】【122†source】.|

### Lưu ý
- Nếu muốn lưu trọng số lên WandB, bạn có thể gọi `wandb.log_artifact()` sau các epoch nhất định để lưu thủ công lên Artifacts. Điều này sẽ giúp quản lý phiên bản trọng số mô hình trên WandB, hữu ích cho việc kiểm tra và so sánh sau này【123†source】【124†source】.

In [ ]:
import wandb
from ultralytics import YOLO

# Khởi tạo dự án WandB
wandb.init(project="ThermalHumanDetect_YOLO")

# Tải mô hình YOLOv8n
model = YOLO("yolov8n.pt")

# Tên mô hình để nhất quán khi lưu trên Kaggle và WandB
model_name = "YOLOvn_ThermalHumanDetect_img640_epochs10_mixedPrecision_withAug_lr0.01"

# Định nghĩa hàm callback để lưu trọng số lên WandB sau mỗi epoch
def log_model_to_wandb(epoch, **kwargs):
    model_path = f"runs/train/{model_name}/weights/epoch_{epoch}.pt"
    artifact = wandb.Artifact(f"{model_name}_weights_epoch_{epoch}", type="model")
    artifact.add_file(model_path)
    wandb.log_artifact(artifact)

# Đăng ký callback cho quá trình huấn luyện
model.add_callback("on_epoch_end", log_model_to_wandb)

# Bắt đầu huấn luyện với callback
model.train(
    data='/kaggle/working/PersonDection-5/data.yaml',
    epochs=10,
    imgsz=640,
    project="ThermalHumanDetect_YOLO",
    name=model_name,
    save=True,
    save_period=1,
    device="cuda",
    half=True,
    lr0=0.01,
    augment=True
)


| **Chỉ Số**        | **Ý Nghĩa**                                                                                          | **Thế nào là tốt?**                                                                                     | **Validation - trước huấn luyện** | **Validation - 3 epochs - Yolov8s - No Augmentation** | **Validation - 15 epochs - Yolov8s - No Augmentation** | **Test - 15 epochs - Yolov8s - No Augmentation** |
|-------------------|------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|-------------------------------|--------------------------|--------------------------|--------------------|
| **Time Training/Inference** | Thời gian huấn luyện hoặc suy luận cho mỗi lần đánh giá mô hình. | Thời gian ngắn giúp tối ưu hiệu suất mô hình. | N/A | 0.08 giờ | 0.4 giờ | 0.01 giờ |
| **Precision**      | Tỉ lệ giữa số dự đoán đúng (true positives) và tổng số các dự đoán (true positives + false positives). Precision cao nghĩa là mô hình ít dự đoán sai. | Precision cao thể hiện độ chính xác của mô hình trong việc đưa ra dự đoán đúng.                          | 0.526                        | 0.808                   | 0.865                   | 0.904             |
| **Recall**         | Tỉ lệ giữa số dự đoán đúng và tổng số đối tượng thực sự có (true positives + false negatives). Recall cao nghĩa là mô hình phát hiện tốt các đối tượng. | Recall cao thể hiện khả năng phát hiện đầy đủ các đối tượng trong dữ liệu.                               | 0.333                        | 0.713                   | 0.811                   | 0.597             |
| **mAP@0.5**        | Mean Average Precision tại ngưỡng IoU = 0.5, đo lường độ chính xác của dự đoán khi yêu cầu độ chồng lấn giữa các hộp giới hạn là 50%. | mAP@0.5 cao cho thấy mô hình có khả năng phát hiện tốt các đối tượng với độ chính xác hộp giới hạn (bounding boxes). | 0.526                        | 0.808                   | 0.898                   | 0.904             |
| **mAP@0.5:0.95**   | Mean Average Precision trung bình qua nhiều ngưỡng IoU từ 0.5 đến 0.95. Chỉ số này phản ánh khả năng dự đoán chính xác với các mức độ chồng lấn khác nhau. | mAP@0.5:0.95 cao thể hiện khả năng tổng quát hóa tốt của mô hình, đặc biệt trong các bài toán có các đối tượng chồng lấn. | 0.333                        | 0.499                   | 0.618                   | 0.597             |



- Trước khi huấn luyện: Khi thực hiện đánh giá ban đầu trên tập validation, mô hình YOLOv8 đạt Precision là 0.526, Recall là 0.333, mAP@0.5 là 0.526, và mAP@0.5:0.95 là 0.333. Đây là các chỉ số từ mô hình YOLOv8 cơ bản chưa được tối ưu hóa.
- Sau khi huấn luyện (3 epochs - Yolov8s - No Augmentation) – 0.08h (4.8min): Sau 3 epochs huấn luyện, các chỉ số cho thấy sự cải thiện đáng kể: Precision tăng lên 0.808, Recall đạt 0.713, mAP@0.5 là 0.808 và mAP@0.5:0.95 là 0.499. Những kết quả này thể hiện rằng mô hình đã học được đặc trưng quan trọng của đối tượng người trong các ảnh nhiệt.
- Sau khi huấn luyện (15 epochs - Yolov8s - No Augmentation) – 0.4h (24min): Sau 15 epochs huấn luyện, các chỉ số tiếp tục cải thiện với Precision đạt 0.865, Recall là 0.811, mAP@0.5 đạt 0.898 và mAP@0.5:0.95 đạt 0.618 trên tập validation. Điều này thể hiện rằng mô hình đã học tốt hơn về khả năng phát hiện và nhận diện đối tượng trong các điều kiện phức tạp.


![img](./img_reports/compare3version_15to30_firstSteps.png)

- YOLOv8n có thể cho kết quả cao hơn trong những bước đầu do cấu trúc nhẹ hơn, cho phép nó học nhanh hơn. Tuy nhiên, về lâu dài, YOLOv8m sẽ dần vượt qua với khả năng tối ưu hóa tốt hơn khi số lượng epochs tăng lên và mô hình hoàn thành các bước điều chỉnh tham số.
- Khi huấn luyện trong thời gian dài (nhiều epochs hơn), ta có thể mong đợi YOLOv8m sẽ vượt qua YOLOv8n nhờ khả năng tổng quát hóa tốt hơn với một lượng lớn tham số.

| **Thuật ngữ** | **Ý nghĩa**                                                                                                   | **Chi tiết**                                                                                                                                                    |
|---------------|----------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Epoch**     | Một lượt hoàn thành toàn bộ tập dữ liệu huấn luyện.                                                            | Mỗi epoch mô hình sẽ xem qua toàn bộ dữ liệu một lần. Số lượng epoch được đặt trước và ảnh hưởng đến mức độ học của mô hình.                                      |
| **Batch**     | Phần nhỏ của tập dữ liệu được sử dụng để huấn luyện trong một lần cập nhật trọng số.                            | Thay vì dùng toàn bộ dữ liệu một lần, dữ liệu được chia thành nhiều batch nhỏ. Kích thước batch (batch size) là số mẫu trong mỗi batch, giúp tiết kiệm bộ nhớ.    |
| **Step**      | Một lần cập nhật trọng số dựa trên một batch.                                                                   | Mỗi step (hoặc iteration) tương ứng với một batch, nơi mô hình tính gradient và cập nhật trọng số. Số step trong một epoch = số mẫu dữ liệu chia cho batch size. |

### Ví dụ
Nếu bạn có 1000 mẫu dữ liệu và batch size là 100, thì sẽ có 10 steps trong mỗi epoch.

| Mối quan hệ                | Công thức                                                       |
|----------------------------|----------------------------------------------------------------|
| Số steps trong một epoch   | steps per epoch = total samples / batch size                  |

- Step có thể đại diện cho epoch nếu số lượng step và epoch là tương đương trong quá trình huấn luyện.
- Nếu step đại diện cho nhiều lần cập nhật (tức là nhiều batch nhỏ trong một epoch), thì có thể số step sẽ nhiều hơn số epoch.

# Finish


| **Chỉ Số**              | **Ý Nghĩa**                                                                                         | **Thế nào là tốt?**                                                                                      | **Validation - trước huấn luyện** | **Validation - 3 epochs - Yolov8s - No Augmentation** | **Validation - 15 epochs - Yolov8s - No Augmentation** | **Test - 15 epochs - Yolov8s - No Augmentation** | **Validation - 30 epochs - Yolov8m - withAug** | **Test - 30 epochs - Yolov8m - withAug** |
|-------------------------|-----------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|----------------------------------|------------------------------------------------------|-----------------------------------------------------|---------------------------------------------------|------------------------------------------------------|----------------------------------------------|
| **Time Training/Inference** | Thời gian huấn luyện hoặc suy luận cho mỗi lần đánh giá mô hình.                                      | Thời gian ngắn giúp tối ưu hiệu suất mô hình.                                                             | N/A                              | 0.08 giờ                                             | 0.4 giờ                                            | 0.01 giờ                                         | 1.5 giờ                                            | 0.02 giờ                                       |
| **Precision**            | Tỉ lệ giữa số dự đoán đúng (true positives) và tổng số các dự đoán (true positives + false positives). Precision cao nghĩa là mô hình ít dự đoán sai. | Precision cao thể hiện độ chính xác của mô hình trong việc đưa ra dự đoán đúng.                           | 0.526                            | 0.808                                                | 0.865                                               | 0.904                                             | 0.912                                               | 0.889                                            |
| **Recall**               | Tỉ lệ giữa số dự đoán đúng và tổng số đối tượng thực sự có (true positives + false negatives). Recall cao nghĩa là mô hình phát hiện tốt các đối tượng. | Recall cao thể hiện khả năng phát hiện đầy đủ các đối tượng trong dữ liệu.                                | 0.333                            | 0.713                                                | 0.811                                               | 0.597                                             | 0.867                                               | 0.829                                            |
| **mAP@0.5**              | Mean Average Precision tại ngưỡng IoU = 0.5, đo lường độ chính xác của dự đoán khi yêu cầu độ chồng lấn giữa các hộp giới hạn là 50%. | mAP@0.5 cao cho thấy mô hình có khả năng phát hiện tốt các đối tượng với độ chính xác hộp giới hạn (bounding boxes). | 0.526                            | 0.808                                                | 0.898                                               | 0.904                                             | 0.921                                               | 0.892                                            |
| **mAP@0.5:0.95**         | Mean Average Precision trung bình qua nhiều ngưỡng IoU từ 0.5 đến 0.95. Chỉ số này phản ánh khả năng dự đoán chính xác với các mức độ chồng lấn khác nhau. | mAP@0.5:0.95 cao thể hiện khả năng tổng quát hóa tốt của mô hình, đặc biệt trong các bài toán có các đối tượng chồng lấn. | 0.333                            | 0.499                                                | 0.618                                               | 0.597                                             | 0.652                                               | 0.628                                            |

